In [98]:
import sqlite3
import pandas as pd

In [99]:
conn = sqlite3.connect('im.db')

In [100]:
df0 = pd.read_sql("""
select primary_title as movie, start_year, genres, region
from movie_basics
join movie_ratings
using (movie_id)
join movie_akas
using (movie_id)
group by movie
having sum(numvotes) > 200000
order by genres asc
;
""", conn)
df0

,movie,start_year,genres,region
0,The Intern,2010,None,None
1,Collide,2010,Action,US
2,Contratiempo,2011,Action,None
3,Kedi,2010,Action,IN
4,Revenge,2011,Action,None
...,...,...,...,...
2201,Dogman,2012,Thriller,US
2202,Friend Request,2013,Thriller,US
2203,Inferno,2010,Thriller,None
2204,The Girl on the Train,2013,Thriller,None


In [101]:
df1 = pd.read_csv('tn.movie_budgets.csv')
df1

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"
...,...,...,...,...,...,...
5777,78,"Dec 31, 2018",Red 11,"$7,000",$0,$0
5778,79,"Apr 2, 1999",Following,"$6,000","$48,482","$240,495"
5779,80,"Jul 13, 2005",Return to the Land of Wonders,"$5,000","$1,338","$1,338"
5780,81,"Sep 29, 2015",A Plague So Pleasant,"$1,400",$0,$0


In [103]:
df2 = df1.merge(df0, how='left', on='movie').drop_duplicates(subset=['movie'])

df2['production_budget'] = df2['production_budget'].str.replace(',', '').str.replace('$', '')
df2['worldwide_gross'] = df2['worldwide_gross'].str.replace(',', '').str.replace('$', '')
df2['worldwide_profit'] = df2['worldwide_gross'].fillna(0).astype('int64') - df2['production_budget'].fillna(0).astype('int64')

df2 = df2.sort_values('worldwide_profit', ascending= False).drop(columns=df2.columns[:2],axis=1)

df2[df2.worldwide_profit > 1000000000].reset_index().head(10)

C:\Users\bkokin\AppData\Local\Temp\ipykernel_6648\2429053443.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df2['production_budget'] = df2['production_budget'].str.replace(',', '').str.replace('$', '')
C:\Users\bkokin\AppData\Local\Temp\ipykernel_6648\2429053443.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df2['worldwide_gross'] = df2['worldwide_gross'].str.replace(',', '').str.replace('$', '')


,index,movie,production_budget,domestic_gross,worldwide_gross,start_year,genres,region,worldwide_profit
0,0,Avatar,425000000,"$760,507,625",2776345279,NaN,NaN,NaN,2351345279
1,42,Titanic,200000000,"$659,363,944",2208208395,NaN,NaN,NaN,2008208395
2,6,Avengers: Infinity War,300000000,"$678,815,482",2048134200,2018.0,"Action,Adventure,Sci-Fi",None,1748134200
3,5,Star Wars Ep. VII: The Force Awakens,306000000,"$936,662,225",2053311220,NaN,NaN,NaN,1747311220
4,33,Jurassic World,215000000,"$652,270,625",1648854864,2015.0,"Action,Adventure,Sci-Fi",None,1433854864
5,66,Furious 7,190000000,"$353,007,020",1518722794,2015.0,"Action,Crime,Thriller",None,1328722794
6,26,The Avengers,225000000,"$623,279,547",1517935897,2012.0,"Action,Adventure,Sci-Fi",None,1292935897
7,260,Harry Potter and the Deathly Hallows: Part II,125000000,"$381,193,157",1341693157,NaN,NaN,NaN,1216693157
8,41,Black Panther,200000000,"$700,059,566",1348258224,2018.0,"Action,Adventure,Sci-Fi",None,1148258224
9,112,Jurassic World: Fallen Kingdom,170000000,"$417,719,760",1305772799,2018.0,"Action,Adventure,Sci-Fi",None,1135772799
